In [1]:
# !pip install tensorflow
# !pip install keras
# !pip install sklearn
# !pip install theano

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import random
from tensorflow import keras
import numpy as np
# from tensorflow.keras import layers, Dense, Input, InputLayer, Flatten
# from tensorflow.keras.models import Sequential, Model


from keras.models import Sequential
from keras import layers
from keras.layers import Dense,Input, InputLayer, Flatten


from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline



from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.optimizers import  adam
from tensorflow.keras.optimizers import SGD,RMSprop, Adam

from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random 
from numpy import *
from PIL import Image
import theano

Using TensorFlow backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
path_test = "./Datasets/training-data/"

In [4]:
CATEGORIES = ['bright_dune', 'crater','dark_dune','impact_ejecta','slope_streak']
# CATEGORIES = ['bright_dune', 'crater','dark_dune','streak']

In [5]:
training = []

def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join(path_test, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (227, 227))
      training.append([new_array, class_num])
      
createTrainingData()

In [6]:
random.shuffle(training)
X = []
y = []

for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, 227, 227, 3)
y = np.array(y)

In [7]:
X.shape

(11919, 227, 227, 3)

In [8]:
X = X.astype('float32')
X /= 255

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [10]:
batch_size = 20
nb_classes = 5
nb_epochs = 10
img_rows, img_columns = 227, 227
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu, input_shape=(227, 227, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(5,  activation=tf.nn.softmax)
])

In [12]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

Train on 9535 samples, validate on 2384 samples
Epoch 1/10
9535/9535 [==============================] - 45s 5ms/sample - loss: 1.1500 - accuracy: 0.5702 - val_loss: 0.8556 - val_accuracy: 0.6963
Epoch 2/10
9535/9535 [==============================] - 39s 4ms/sample - loss: 0.7941 - accuracy: 0.7127 - val_loss: 0.7393 - val_accuracy: 0.7404
Epoch 3/10
9535/9535 [==============================] - 40s 4ms/sample - loss: 0.6039 - accuracy: 0.7892 - val_loss: 0.6192 - val_accuracy: 0.7773
Epoch 4/10
9535/9535 [==============================] - 39s 4ms/sample - loss: 0.4715 - accuracy: 0.8314 - val_loss: 0.5740 - val_accuracy: 0.8049
Epoch 5/10
9535/9535 [==============================] - 40s 4ms/sample - loss: 0.3354 - accuracy: 0.8805 - val_loss: 0.6201 - val_accuracy: 0.8121
Epoch 6/10
9535/9535 [==============================] - 40s 4ms/sample - loss: 0.2667 - accuracy: 0.9115 - val_loss: 0.7202 - val_accuracy: 0.8146
Epoch 7/10
9535/9535 [==============================] - 40s 4ms/sample

In [14]:
def findClass(image_path):
    image_1 = cv2.imread(image_path)
    image_1 = cv2.resize(image_1, (227, 227))
    np.array(image_1).reshape(-1, 227, 227, 3)
    image_1 = image_1.astype('float32')
    image_1 /= 255 
    prediction = model.predict(np.array([image_1], np.float32))
    print(prediction)
    return CATEGORIES[np.argmax(prediction)]

In [15]:
testData = [
    r'./Datasets/test-data/bright_dune/ESP_016576_2570_RED-0061.jpg', 
    r'./Datasets/test-data/dark_dune/ESP_018334_2600_RED-0501.jpg',
    r'./Datasets/test-data/streak/ESP_011289_1950_RED-0067.jpg', 
    r'./Datasets/hirise-map-proj-v3_2/slope_streak/ESP_011289_1950_RED-0230.jpg',
    r'./Datasets/hirise-map-proj-v3_2/crater/ESP_011425_1775_RED-0030-r90.jpg'
]

# findClass(r'./Datasets/test-data/bright_dune/ESP_016576_2570_RED-0061.jpg')

for i in testData:
    print(findClass(i))

# CATEGORIES = ['bright_dune', 'crater','dark_dune','impact','streak']

[[6.97488649e-05 1.26305795e-05 3.57039176e-09 7.95030600e-13
  9.99917626e-01]]
slope_streak
[[1.1492472e-08 2.7434713e-01 7.0048183e-01 5.6074242e-07 2.5170470e-02]]
dark_dune
[[0.0322766  0.00528794 0.01380907 0.00782056 0.94080573]]
slope_streak
[[2.9163374e-02 2.4930069e-03 4.6472196e-03 3.7592159e-05 9.6365887e-01]]
slope_streak
[[3.1796783e-06 9.9924976e-01 1.0500467e-04 7.0692120e-05 5.7131908e-04]]
crater


In [2]:
score = model.evaluate(X_test, y_test, verbose = 0 )
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Test Loss: 0.856600800216598
Test Accuracy: 0.8183725


In [17]:
import pickle

In [19]:
# creates a HDF5 file 'my_model.h5'
model.save('models/cnn.h5')
# pickle.dump( model, open("classification_model_v1.pickle", "wb" ))